In [1]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

In [4]:
from algorithm_utils import *
import general_utils as gu

In [5]:
testing_length = 200
start_index = 5000
pair_df: pd.DataFrame = gu.load_local_data().iloc[start_index:start_index+testing_length].reset_index(drop=True)

In [6]:
zigzag_df = zigzag(pair_df)
h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
bos_df = find_confirmed_boss(pbos_df, pair_df)
lpl_df = find_lpls(bos_df, zigzag_df)
lplb_df = find_lplbs(bos_df, zigzag_df)

In [10]:
pt = gu.PlottingTool()
pt.draw_candlesticks(pair_df)
pt.draw_zigzag(zigzag_df)
# pt.draw_zigzag(h_o_zigzag_df, title='Higher order Zigzag', color='black')
# pt.draw_points_with_label(pbos_df, "PBOS")
pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "peak"], "BOS-L", color="red")
pt.draw_points_with_label(lplb_df.loc[lplb_df.pivot_type == "valley"], "LPLB-L", color="orange")
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="blue")
pt.show()